In [2]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np


mid = MidiFile('C:/Users/Aaqui/Desktop/magenta/Database/Beethovan/furelise.mid') 
#mid2 = MidiFile('C:/Users/Aaqui/Desktop/magenta/Database/Beethovan/brondo.mid') 
notes = []

notes = []
for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1])

        #for second song input
#for msg in mid2:
#    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
 #       data = msg.bytes()
 #       notes.append(data[1])
    
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))


notes = [list(note) for note in notes]

X = []
y = []
n_prev = 30
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
X_test = X[-200:] #for prediction
X = X[:-200]
y = y[:-200]


#model description/////////////////////////////////////////////////////////
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)


filepath="C:/Users/Aaqui/Untitled Folder/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', period=5)

#model testing/////////////////////////////////////////////////////////////
model.fit(np.array(X), np.array(y), 32, 20, verbose=1, callbacks=[model_save_callback])
model.save('C:/Users/Aaqui/Untitled Folder/Keras based/weights/weights1.h5')


#prediction part////////////////////////////////////////////////////////////
from keras.models import load_model as ld
trained_model = ld('C:/Users/Aaqui/Untitled Folder/Keras based/weights/weights1.h5')
prediction = trained_model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

#saving to MIDI////////////////////////////////////////////////////////
mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velocity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('LSTM_music1.mid')

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\Aaqui\Anaconda3\envs\PythonCPU\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1024/1024 [==============================] - 7s 6ms/step - loss: 0.0337
Epoch 2/20
1024/1024 [==============================] - 3s 3ms/step - loss: 0.0274
Epoch 3/20
1024/1024 [==============================] - 3s 3ms/step - loss: 0.0278
Epoch 4/20
1024/1024 [==============================] - 3s 3ms/step - loss: 0.0284
Epoch 5/20
1024/1024 [==============================] - 3s 3ms/step - loss: 0.0261

Epoch 00005: saving model to C:/Users/Aaqui/Untitled Folder/checkpoint_model_05.hdf5
Epoch 6/20
1024/1024 [==============================] - 3s 3ms/step - loss: 0.0256
Epoch 7/20
1024/1024 [==============================] - 3s 3ms/step - loss: 0.0251
Epoch 8/20
1024/1024 [==============================] - 3s 3ms/step - loss: 0.0255
Epoch 9/20
1024/1024 [==============================] - 3s 3ms/step - loss: 0.0253
Epoc